<a href="https://colab.research.google.com/github/LimJH27/LJH/blob/master/translation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 로드/전처리

In [ ]:
import os
import re
import shutil
import zipfile

import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import urllib3
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
#####################################################
#        한/영 데이터 변수 선언 & 경로 지정         #
#####################################################

# with with open(path_to_file, "r") as f:
#     raw = f.read().splitlines()

# print("Data Size: ", len(raw))
# print("Example :")

# for sen in raw[0:100][::20]: print(">>", sen) 


FileNotFoundError: ignored

In [ ]:
#####################################################
#              기존 영/프 데이터 셀                 #
#####################################################

num_samples = 33000

http = urllib3.PoolManager()
url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)
path_to_zip = tf.keras.utils.get_file('spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip', extract=True) # 추가
path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"


num_samples = 33000

In [ ]:
def to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

In [ ]:
def preprocess_sentence(sent):
  # 위에서 구현한 함수를 내부적으로 호출
  sent = to_ascii(sent.lower())

  # 단어와 구두점 사이에 공백 추가.
  # ex) "I am a student." => "I am a student ."
  sent = re.sub(r"([?.!,¿])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
  sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

  sent = re.sub(r"\s+", " ", sent)
  return sent

In [ ]:
#####################################################
#   en/fr_sentence -> ko/en_sentence 수정 완료      #
#####################################################

# 전처리 테스트
ko_sent = u"Have you had dinner?"
en_sent = u"Avez-vous déjà diné?"
print(preprocess_sentence(ko_sent))
print(preprocess_sentence(en_sent).encode('utf-8'))

have you had dinner ?
b'avez vous deja dine ?'


In [ ]:
#####################################################
#       fra.txt 경로를 새데이터 경로로 수정         #
#####################################################

def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  with open("fra.txt", "r") as lines:
    for i, line in enumerate(lines):
      # source 데이터와 target 데이터 분리
      src_line, tar_line, _ = line.strip().split('\t')

      # source 데이터 전처리
      src_line = [w for w in preprocess_sentence(src_line).split()]

      # target 데이터 전처리
      tar_line = preprocess_sentence(tar_line)
      tar_line_in = [w for w in ("<sos> " + tar_line).split()]
      tar_line_out = [w for w in (tar_line + " <eos>").split()]

      encoder_input.append(src_line)
      decoder_input.append(tar_line_in)
      decoder_target.append(tar_line_out)

      if i == num_samples - 1:
        break

  return encoder_input, decoder_input, decoder_target

In [ ]:
#####################################################
#              en/fra -> ko/en 수정 완료            #
#####################################################

sents_ko_in, sents_en_in, sents_en_out = load_preprocessed_data()  # sents_ko_out => sents_en_out 으로 변경 / 한영 번역기용 아니고 당장 돌려보기 용
print(sents_ko_in[:5])
print(sents_en_in[:5])
print(sents_en_out[:5])

[['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.'], ['hi', '.']]
[['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!'], ['<sos>', 'salut', '.']]
[['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>'], ['salut', '.', '<eos>']]


In [ ]:
#####################################################
#              en/fra -> ko/en 수정 완료            #
#####################################################

tokenizer_ko = Tokenizer(filters="", lower=False)
tokenizer_ko.fit_on_texts(sents_ko_in)
encoder_input = tokenizer_ko.texts_to_sequences(sents_ko_in)
encoder_input = pad_sequences(encoder_input, padding="post")

tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
tokenizer_en.fit_on_texts(sents_en_out)

decoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_en.texts_to_sequences(sents_en_out)
decoder_target = pad_sequences(decoder_target, padding="post")

In [ ]:
#####################################################
#              en/fra -> ko/en 수정 완료            #
#####################################################

src_vocab_size = len(tokenizer_ko.word_index) + 1
tar_vocab_size = len(tokenizer_en.word_index) + 1
print("한국어 단어 집합의 크기 : {:d}, 영어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 4635, 프랑스어 단어 집합의 크기 : 8117


In [ ]:
#####################################################
#              en/fra -> ko/en 수정 완료            #
#####################################################

src_to_index = tokenizer_ko.word_index
index_to_src = tokenizer_ko.index_word
tar_to_index = tokenizer_en.word_index
index_to_tar = tokenizer_en.index_word

In [ ]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[12341 11391 23738 ... 18244 26379 29232]


In [ ]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [ ]:
encoder_input[30997]

array([   2,   65, 2599,    1,    0,    0,    0,    0], dtype=int32)

In [ ]:
decoder_input[30997]

array([   2,   12,   16,   61, 3737,    1,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0], dtype=int32)

In [ ]:
decoder_target[30997]

array([  12,   16,   61, 3737,    1,    3,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0], dtype=int32)

In [ ]:
n_of_val = int(33000*0.1)
print(n_of_val)

3300


In [ ]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [ ]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (29700, 8)
훈련 target 데이터의 크기 : (29700, 16)
훈련 target 레이블의 크기 : (29700, 16)
테스트 source 데이터의 크기 : (3300, 8)
테스트 target 데이터의 크기 : (3300, 16)
테스트 target 레이블의 크기 : (3300, 16)


# 전처리 끝, 번역기 만들기

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [ ]:
# 임베딩 벡터와 LSTM 은닉 상태 크기 고정
latent_dim = 50

In [ ]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(src_vocab_size, latent_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # Masking : 패딩토큰인 숫자 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [ ]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# 출력층에 softmax -> categorical_crossentropy
# 원-핫 인코딩을 하지 않은 상태로 정수 레이블에 대해 다중 클래스 분류 시 loss function에 sparse_categorical_crossentropy을 사용.
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 50)     231750      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 50)     405850      ['input_2[0][0]']                
                                                                                              

In [ ]:
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 128, epochs = 50)

Epoch 1/50
233/233 [==============================] - 114s 424ms/step - loss: 3.1159 - acc: 0.6225 - val_loss: 1.8470 - val_acc: 0.6759
Epoch 2/50
233/233 [==============================] - 94s 405ms/step - loss: 1.6602 - acc: 0.7406 - val_loss: 1.5984 - val_acc: 0.7498
Epoch 3/50
233/233 [==============================] - 94s 405ms/step - loss: 1.4854 - acc: 0.7586 - val_loss: 1.4641 - val_acc: 0.7651
Epoch 4/50
233/233 [==============================] - 94s 405ms/step - loss: 1.3973 - acc: 0.7692 - val_loss: 1.3873 - val_acc: 0.7753
Epoch 5/50
233/233 [==============================] - 94s 405ms/step - loss: 1.3162 - acc: 0.7844 - val_loss: 1.3117 - val_acc: 0.7912
Epoch 6/50
233/233 [==============================] - 94s 404ms/step - loss: 1.2438 - acc: 0.7969 - val_loss: 1.2483 - val_acc: 0.8001
Epoch 7/50
233/233 [==============================] - 94s 405ms/step - loss: 1.1871 - acc: 0.8056 - val_loss: 1.2050 - val_acc: 0.8070
Epoch 8/50
233/233 [==============================] - 

## seq2seq 기계 번역기

In [ ]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

In [ ]:
# 디코더
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2= dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

In [ ]:
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
    if (sampled_char == '<eos>' or
        len(decoded_sentence) > 50):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [ ]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence

In [ ]:
# 테스트

for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

입력문장 : what will we eat ? 
정답문장 : qu allons nous manger ? 
번역문장 : que nous sommes demain ? 
--------------------------------------------------
입력문장 : a comma is missing . 
정답문장 : il manque une virgule . 
번역문장 : il manque une fois . 
--------------------------------------------------
입력문장 : i m sort of tired . 
정답문장 : je suis en quelque sorte fatiguee . 
번역문장 : je suis en quelque chose suis tres malade . 
--------------------------------------------------
입력문장 : he s a big coward . 
정답문장 : c est un grand lache . 
번역문장 : c est un un homme . 
--------------------------------------------------
입력문장 : you re not dead . 
정답문장 : vous n etes pas morte . 
번역문장 : vous n etes pas mort . 
--------------------------------------------------


In [ ]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

입력문장 : tom s trapped . 
정답문장 : tom est piege . 
번역문장 : tom est . 
--------------------------------------------------
입력문장 : what time is it ? 
정답문장 : quelle heure est il ? 
번역문장 : quel est l qui de l heure ? 
--------------------------------------------------
입력문장 : tom s a doctor . 
정답문장 : tom est medecin . 
번역문장 : tom est un peu . 
--------------------------------------------------
입력문장 : aren t you happy ? 
정답문장 : n etes vous pas heureux ? 
번역문장 : ne vous etes pas n train de les yeux ? 
--------------------------------------------------
입력문장 : hug me tight . 
정답문장 : serre moi bien fort . 
번역문장 : viens vous en securite . 
--------------------------------------------------


# Attention 시각화




![](https://aiffelstaticprd.blob.core.windows.net/media/original_images/GN-4-L-9.jpg)

참고 :     
[1](https://tutorials.pytorch.kr/intermediate/seq2seq_translation_tutorial.html)

[2](https://colab.research.google.com/drive/1arlT6YVpUVnmZAbsu3e4sHqbk7c0cu8A#scrollTo=2FBtAEqIkIpQ) - 훈련하기 2.train_step